In [17]:
import cv2
import numpy as np
import math

image_path = "DamasconeA/DamasconeA.png"
image = cv2.imread(image_path)
if image is None:
    print(f"[ERROR] Could not read image from {image_path}")
    exit()

start_x, start_y = 340, 80  # starting point (x, y)
end_x, end_y = 600, 300  # ending point (x, y)

# Crop the image using the defined coordinates
cropped_image = image[start_y:end_y, start_x:end_x]
image = cv2.resize(cropped_image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
def compute_marker_angle(marker_corners):
    # Define the corners
    top_left = marker_corners[0]
    top_right = marker_corners[1]
    bottom_right = marker_corners[2]
    bottom_left = marker_corners[3]

    # Horizontal angle: mean of top-left to top-right and bottom-left to bottom-right
    angleh_top = math.degrees(math.atan2(top_right[1] - top_left[1], top_right[0] - top_left[0]))
    angleh_bottom = math.degrees(math.atan2(bottom_right[1] - bottom_left[1], bottom_right[0] - bottom_left[0]))
    angleh = (angleh_top + angleh_bottom) / 2

    # Vertical angle: mean of top-left to bottom-left and top-right to bottom-right
    anglev_left = math.degrees(math.atan2(bottom_left[1] - top_left[1], bottom_left[0] - top_left[0]))
    anglev_right = math.degrees(math.atan2(bottom_right[1] - top_right[1], bottom_right[0] - top_right[0]))
    anglev = (anglev_left + anglev_right) / 2

    return angleh, anglev


aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_ARUCO_ORIGINAL)
arucoParams = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, arucoParams)
(corners, ids, _) = detector.detectMarkers(image)

if len(corners) > 0:
    marker_corners = corners[0][0]
    angleh, anglev = compute_marker_angle(marker_corners)
    print(f"[INFO] Marker angle horizontally: {angleh:.2f} degrees")
    print(f"[INFO] Marker angle vertically: {anglev:.2f} degrees")
else:
    print("[ERROR] No ArUco markers detected!")
    exit()

[INFO] Marker angle horizontally: -89.11 degrees
[INFO] Marker angle vertically: 0.00 degrees


In [11]:
def create_marker_mask(image, marker_corners, padding=10):
    mask = np.ones_like(image) * 255
    if marker_corners is not None and len(marker_corners) > 0:
        x_min = int(max(0, np.min(marker_corners[:, 0]) - padding))
        y_min = int(max(0, np.min(marker_corners[:, 1]) - padding))
        x_max = int(min(image.shape[1], np.max(marker_corners[:, 0]) + padding))
        y_max = int(min(image.shape[0], np.max(marker_corners[:, 1]) + padding))
        cv2.rectangle(mask, (x_min, y_min), (x_max, y_max), 0, -1)
    return mask


# start_x, start_y = 60, 0  # starting point (x, y)
# object_image = image[start_y:, start_x:]
#
# cv2.imshow("Image", object_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

def merge_close_points(points, threshold=5):
    if len(points) == 0:
        return np.empty((0, 2))
    merged_points = []
    points = sorted(points, key=lambda p: (p[0], p[1]))
    while points:
        ref = points.pop(0)
        close_points = [ref]
        remaining = []
        for pt in points:
            if np.linalg.norm(np.array(ref) - np.array(pt)) < threshold:
                close_points.append(pt)
            else:
                remaining.append(pt)
        merged_points.append(np.mean(close_points, axis=0))
        points = remaining
    return np.array(merged_points)


def detect_colored_circles(image, marker_corners=None):
    b, g, r = cv2.split(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    mask = create_marker_mask(gray, marker_corners)
    gray_masked = cv2.bitwise_and(gray, mask)
    b_masked = cv2.bitwise_and(b, mask)
    g_masked = cv2.bitwise_and(g, mask)
    r_masked = cv2.bitwise_and(r, mask)

    params = cv2.SimpleBlobDetector_Params()
    params.filterByArea = True
    params.minArea = 5
    params.maxArea = 2000
    params.filterByConvexity = True
    params.minConvexity = 0.5

    detector = cv2.SimpleBlobDetector_create(params)
    all_points = []
    for channel in [b_masked, g_masked, r_masked, gray_masked]:
        keypoints = detector.detect(channel)
        all_points.extend([kp.pt for kp in keypoints])
    unique_points = merge_close_points(all_points, threshold=5)
    return unique_points


all_circles = detect_colored_circles(image, marker_corners)
if len(all_circles) == 0:
    print("[ERROR] No circles detected!")
    exit()

print(f"[INFO] Detected {len(all_circles)} circles after filtering.")

[INFO] Detected 25 circles after filtering.


In [12]:
import numpy as np
import cv2
from scipy.stats import linregress

# Assuming `image` is your background image where you want to plot everything
vis = image.copy()  # Create a copy of the image to draw on

# To store the angles for rows and columns
row_angles = []
col_angles = []

# Step 1: Sort points by y to group them into rows (we'll split into 5 rows as the grid is 5x5)
sorted_by_y = all_circles[np.argsort(all_circles[:, 1])]

# Step 2: Group points into rows based on their y-values
rows = np.split(sorted_by_y, 5)

# Step 3: Perform linear regression on each row (horizontal lines)
for row in rows:
    # Sort each row by x-coordinate (to ensure the points are in horizontal order)
    sorted_row = row[np.argsort(row[:, 0])]

    # Perform linear regression for the entire row (x vs y)
    slope, intercept, _, _, _ = linregress(sorted_row[:, 0], sorted_row[:, 1])

    # Generate the x values for drawing the regression line (using min and max of x)
    x_vals = np.array([min(sorted_row[:, 0]), max(sorted_row[:, 0])])
    y_vals = slope * x_vals + intercept

    angle_rad = math.atan(slope)
    angle_deg = math.degrees(angle_rad)
    row_angles.append(angle_deg)

    # Draw the regression line on the image (color: blue for rows)
    start_point = (int(x_vals[0]), int(y_vals[0]))
    end_point = (int(x_vals[1]), int(y_vals[1]))
    cv2.line(vis, start_point, end_point, (255, 0, 0), 2)  # Blue line for rows

# Step 4: Sort points by x to group them into columns
# First, sort points based on x-coordinate to ensure correct column grouping
sorted_by_x = all_circles[np.argsort(all_circles[:, 0])]

# Step 5: Group points into columns based on their x-values
columns = np.split(sorted_by_x, 5)

# Step 6: Perform linear regression on each column (vertical lines)
for col in columns:
    # Sort each column by y-coordinate (to ensure they are in vertical order)
    sorted_col = col[np.argsort(col[:, 1])]

    # Perform linear regression for the entire column (x vs y)
    slope, intercept, _, _, _ = linregress(sorted_col[:, 1], sorted_col[:, 0])  # Reversed axes for vertical regression

    # Generate the y values for drawing the regression line (using min and max of y)
    y_vals = np.array([min(sorted_col[:, 1]), max(sorted_col[:, 1])])
    x_vals = slope * y_vals + intercept  # Solve for x from y = mx + b (vertical line equation)

    angle_rad = math.atan(slope)
    angle_deg = math.degrees(angle_rad)
    col_angles.append(angle_deg)

    # Draw the regression line on the image (color: green for columns)
    start_point = (int(x_vals[0]), int(y_vals[0]))
    end_point = (int(x_vals[1]), int(y_vals[1]))
    cv2.line(vis, start_point, end_point, (0, 255, 0), 2)  # Green line for columns

# Step 7: Draw circles on the points
for pt in all_circles:
    cv2.circle(vis, (int(pt[0]), int(pt[1])), 3, (128, 128, 128), -1)

marker_corners = marker_corners.astype(np.int32)
for i in range(4):
    cv2.line(vis, tuple(marker_corners[i]), tuple(marker_corners[(i + 1) % 4]), (0, 255, 0), 1)

# Calculate the mean angles
mean_row_angle = np.mean(row_angles) - 90
mean_col_angle = np.mean(col_angles)

# Calculate the overall mean angle
mean_angle = np.mean([mean_row_angle, mean_col_angle])

# Print the results
print(f"Marker angle horizontally: {angleh:.2f} degrees")
print(f"Marker angle vertically: {anglev:.2f} degrees")
print(f"Mean angle for rows: {mean_row_angle:.2f} degrees")
print(f"Mean angle for columns: {mean_col_angle:.2f} degrees")
# print(f"Overall mean angle: {mean_angle:.2f} degrees")

print(mean_row_angle - angleh)
print(mean_col_angle - anglev)
print(np.mean([mean_row_angle - angleh, mean_col_angle - anglev]))

# Display the image
cv2.imshow("Detected Circles and Lines", vis)
cv2.waitKey(0)
cv2.destroyAllWindows()


Marker angle horizontally: -89.11 degrees
Marker angle vertically: 0.00 degrees
Mean angle for rows: -89.61 degrees
Mean angle for columns: 1.14 degrees
-0.5045025079893577
1.1360486073460705
0.3157730496783564


In [14]:
import cv2
import numpy as np

# Load the image
image_path = "DamasconeA/DamasconeA.png"
image = cv2.imread(image_path)
if image is None:
    print(f"[ERROR] Could not read image from {image_path}")
    exit()

# Define the ArUco dictionary and parameters
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_ARUCO_ORIGINAL)
aruco_params = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)

# Detect the markers
corners, ids, _ = detector.detectMarkers(image)

# Global variables for transformation
global_rvec = None
global_tvec = None
global_camera_matrix = None
global_dist_coeffs = None
result_image = None

# Storage for clicked points
clicked_points = []


def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Store the clicked point
        clicked_points.append((x, y))
        display_image = result_image.copy()

        # Prepare the image point for undistortion.
        image_point = np.array([[[x, y]]], dtype=np.float32)

        # Undistort the point using the projection matrix to obtain pixel coordinates.
        undistorted_point_px = cv2.undistortPoints(
            image_point,
            global_camera_matrix,
            global_dist_coeffs,
            P=global_camera_matrix
        )
        undistorted_point_px = undistorted_point_px[0, 0]

        # Convert undistorted pixel coordinates into normalized coordinates.
        fx = global_camera_matrix[0, 0]
        fy = global_camera_matrix[1, 1]
        cx = global_camera_matrix[0, 2]
        cy = global_camera_matrix[1, 2]
        x_norm = (undistorted_point_px[0] - cx) / fx
        y_norm = (undistorted_point_px[1] - cy) / fy

        # Construct the ray in camera coordinates (normalized)
        ray = np.array([x_norm, y_norm, 1.0])

        # Transform to marker coordinates:
        rotation_matrix, _ = cv2.Rodrigues(global_rvec)
        rotation_matrix_inv = np.linalg.inv(rotation_matrix)

        # Compute the camera center in marker coordinates and flatten to 1D array.
        cam_to_marker = np.dot(rotation_matrix_inv, -global_tvec).flatten()

        # Transform the ray direction into the marker coordinate frame.
        ray_in_marker = np.dot(rotation_matrix_inv, ray)

        # Compute the intersection scale factor for ray with the z=0 marker plane.
        if ray_in_marker[2] != 0:
            scale = -cam_to_marker[2] / ray_in_marker[2]
        else:
            scale = 0

        # Calculate the 3D intersection point in the marker frame.
        point_3d = cam_to_marker + scale * ray_in_marker

        # Debug prints (for verification)
        print("Undistorted pixel coordinates:", undistorted_point_px)
        print("Normalized image coordinates:", x_norm, y_norm)
        print("Ray in camera coordinates:", ray)
        print("Camera center in marker coordinates (flattened):", cam_to_marker)
        print("Ray in marker coordinates:", ray_in_marker)
        print("Scale factor:", scale)
        print("Intersection point in marker coordinates:", point_3d)

        # Extract physical coordinates (x and y)
        x_coord = point_3d.item(0)
        y_coord = point_3d.item(1)

        # Draw the clicked point and display coordinates
        cv2.circle(display_image, (x, y), 5, (0, 0, 255), -1)
        text = f"Image: ({x}, {y}), Physical: ({x_coord:.1f}, {y_coord:.1f}) mm"
        cv2.putText(display_image, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        for px, py in clicked_points:
            cv2.circle(display_image, (px, py), 3, (0, 255, 255), -1)

        cv2.imshow("Pose Estimation", display_image)



if ids is not None:
    # Define marker size (mm)
    marker_size = 16

    # Camera intrinsic matrix
    global_camera_matrix = np.array([
        [606.9752807617188, 0.0, 431.1286926269531],
        [0.0, 606.9803466796875, 242.2771759033203],
        [0.0, 0.0, 1.0]
    ], dtype=np.float32)

    # Distortion coefficients
    global_dist_coeffs = np.zeros((5,), dtype=np.float32)

    # Process first marker
    marker_corners = corners[0][0]

    # Define 3D coordinates with desired orientation
    objPoints = np.array([
        [marker_size/2, -marker_size/2, 0.0],   # top-left
        [-marker_size/2, -marker_size/2, 0.0],  # bottom-left
        [-marker_size/2, marker_size/2, 0.0],   # bottom-right
        [marker_size/2, marker_size/2, 0.0]     # top-right
    ], dtype=np.float32)

    print(objPoints)
    print(marker_corners)

    # Solve for pose
    success, global_rvec, global_tvec = cv2.solvePnP(
        objPoints,
        marker_corners,
        global_camera_matrix,
        global_dist_coeffs
    )

    if success:
        # Make a copy for display
        result_image = image.copy()

        # Draw axes and marker outline
        cv2.drawFrameAxes(result_image, global_camera_matrix, global_dist_coeffs,
                          global_rvec, global_tvec, marker_size * 3, 1)

        cv2.polylines(
            result_image,
            [marker_corners.astype(np.int32)],
            True,
            (0, 255, 0),
            1
        )

        # Show initial result
        cv2.imshow("Pose Estimation", result_image)

        # Set up mouse callback
        cv2.setMouseCallback("Pose Estimation", mouse_callback)

        print("Click on points to see physical coordinates (mm). Press any key to exit.")
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("[ERROR] Failed to determine marker pose!")
else:
    print("[ERROR] No ArUco markers detected!")

[[ 8. -8.  0.]
 [-8. -8.  0.]
 [-8.  8.  0.]
 [ 8.  8.  0.]]
[[351. 103.]
 [351.  87.]
 [368.  87.]
 [368. 103.]]
Click on points to see physical coordinates (mm). Press any key to exit.
Undistorted pixel coordinates: [473. 191.]
Normalized image coordinates: 0.06898355 -0.08447914
Ray in camera coordinates: [ 0.06898355 -0.08447914  1.        ]
Camera center in marker coordinates (flattened): [234.0490673   38.27184483 545.03884138]
Ray in marker coordinates: [-0.24821436  0.11820667 -0.9676323 ]
Scale factor: 563.2706155423386
Intersection point in marker coordinates: [ 94.2372112  104.85418887   0.        ]


In [18]:
# Calculate the mean position of all detected circle centers

# Calculate the mean position of all detected circle centers
mean_x = np.mean(all_circles[:, 0])
mean_y = np.mean(all_circles[:, 1])

# Apply inverse transformation (scale down by 2x, then shift by crop offset)
original_mean_x = (mean_x / 2) + start_x
original_mean_y = (mean_y / 2) + start_y
print(f"Original image coordinates before transform: ({original_mean_x:.2f}, {original_mean_y:.2f})")

# Use these coordinates with the perspective transformation algorithm
image_point = np.array([[[original_mean_x, original_mean_y]]], dtype=np.float32)

# Undistort the point using the projection matrix to obtain pixel coordinates
undistorted_point_px = cv2.undistortPoints(
    image_point,
    global_camera_matrix,
    global_dist_coeffs,
    P=global_camera_matrix
)
undistorted_point_px = undistorted_point_px[0, 0]

# Continue with the rest of the algorithm...
fx = global_camera_matrix[0, 0]
fy = global_camera_matrix[1, 1]
cx = global_camera_matrix[0, 2]
cy = global_camera_matrix[1, 2]
x_norm = (undistorted_point_px[0] - cx) / fx
y_norm = (undistorted_point_px[1] - cy) / fy

# Calculate real-world coordinates as before
ray = np.array([x_norm, y_norm, 1.0])
rotation_matrix, _ = cv2.Rodrigues(global_rvec)
rotation_matrix_inv = np.linalg.inv(rotation_matrix)
cam_to_marker = np.dot(rotation_matrix_inv, -global_tvec).flatten()
ray_in_marker = np.dot(rotation_matrix_inv, ray)

if ray_in_marker[2] != 0:
    scale = -cam_to_marker[2] / ray_in_marker[2]
else:
    scale = 0

point_3d = cam_to_marker + scale * ray_in_marker
real_x = point_3d.item(0)
real_y = point_3d.item(1)

print(f"Mean position in real-world coordinates: ({real_x:.2f}, {real_y:.2f}) mm")
# Visualize the mean position on the image
vis_mean = image.copy()
cv2.circle(vis_mean, (int(mean_x), int(mean_y)), 8, (0, 0, 255), -1)
cv2.putText(vis_mean, f"Mean: ({real_x:.1f}, {real_y:.1f}) mm",
            (int(mean_x) + 10, int(mean_y) + 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

cv2.imshow("Mean Circle Position", vis_mean)
cv2.waitKey(0)
cv2.destroyAllWindows()

Original image coordinates before transform: (472.69, 191.90)
Mean position in real-world coordinates: (95.09, 104.54) mm
